In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import f1_score,average_precision_score
import lightgbm as lgb
import catboost as cb
from sklearn.model_selection import StratifiedKFold
import warnings
warnings.filterwarnings('ignore')

train_x = pd.read_csv('creditcard_train.csv')
test_x = pd.read_csv('creditcard_test.csv')
res = test_x[['Index']]
del train_x['Index']
del test_x['Index']

test_x['Class'] = -1
data = pd.concat([train_x,test_x],axis=0,ignore_index=True)
train_index = data[data['Class']!=-1].index.tolist()
test_index = data[data['Class']==-1].index.tolist()
data['Time'] = data['Time'].astype(int)
data['Day'] = (data['Time']//(3600*24)).astype(int)
data['Hour'] = ((data['Time']-data['Day']*24*3600)//(3600)).astype(int)



def get_pywt_1(x):
    import pywt
    db1 = pywt.Wavelet('db1')
    A= pywt.wavedec(x, db1,level=4)
    return A[0][0]
def get_pywt_2(x):
    import pywt
    db1 = pywt.Wavelet('db1')
    A= pywt.wavedec(x, db1,level=4)
    return A[0][1]
def get_pywt_3(x):
    import pywt
    db1 = pywt.Wavelet('db1')
    A= pywt.wavedec(x, db1,level=4)
    return A[1][0]
def get_pywt_4(x):
    import pywt
    db1 = pywt.Wavelet('db1')
    A= pywt.wavedec(x, db1,level=4)
    return A[1][1]
columns=['V'+str(i) for i in range(1,29)]
data['xiao1']=data[columns].apply(get_pywt_1,axis=1)
data['xiao2']=data[columns].apply(get_pywt_2,axis=1)
data['xiao3']=data[columns].apply(get_pywt_3,axis=1)
data['xiao4']=data[columns].apply(get_pywt_4,axis=1)


pca_cols = []
for i in range(1,29):
    pca_cols.append('V'+str(i))
def getFeature(x):
    return ' '.join(x.astype(str).values.tolist())
se  = data[pca_cols].apply(getFeature,axis=1)
st = pd.Series(se.drop_duplicates().values)

data['Id'] = se.map(pd.Series(st.index,index=st.values))

for co in ['Id','Amount']:
    data[co+'_ount'] = data[co].map(data[co].value_counts())

train_x = data.drop(['Id'],axis=1).loc[train_index].reset_index(drop=True)
test_x = data.drop(['Id'],axis=1).loc[test_index].reset_index(drop=True)
train_y = train_x.pop('Class')




del test_x['Class']

In [6]:
params_initial_lgb = {
        'num_leaves':31, 
        'learning_rate':0.01, 
    'boosting':'gbdt',
    'min_child_samples':10,

    'bagging_fraction':0.7, 
    'bagging_freq':1,
    'feature_fraction':0.7, 

    'reg_alpha':0,
    'reg_lambda':1, 
    'metric':'binary_logloss',
    'objective':'binary'
}
NBR = 20000
VBE = 100
ESR = 200
def my_score(preds,data_vali):
    labels = data_vali.get_label()
    return 'AP',average_precision_score(labels, preds),True
def searchBestCut(ytrue,ypre):
    lst = []
    for i in range(10,81,1):
        lst.append(f1_score(ytrue,(pd.Series(ypre)>=i*0.01).astype(int).values))
    se = pd.Series(lst,index=range(10,81,1)).sort_values()
    return (se.index[-1])*0.01,se.values[-1]
print(train_x.shape)
print(test_x.shape)
prob = pd.DataFrame()
cate = pd.DataFrame()
num = 0
apscore = []
fscore = []

X = np.array(train_x)
Y = np.array(train_y)
X_test = np.array(test_x)

skf = StratifiedKFold(n_splits=5,random_state=2020,shuffle=True)
for train_part_index,evals_index in skf.split(train_x,train_y):
    EVAL_RESULT = {}
    X_train, X_val, y_train, y_val = X[train_part_index], X[evals_index], Y[train_part_index], Y[evals_index]

    model = cb.CatBoostClassifier( iterations=500,
                                   depth = 8,
                                   learning_rate = 0.1,
                                   custom_loss='AUC',
                                   eval_metric='AUC',
                                   bagging_temperature=0.83,
                                   od_type='Iter',
                                   rsm = 0.8,
                                   od_wait= 100,
                                   metric_period = 50,
                                   l2_leaf_reg = 5,
                                   thread_count = 50,
                                   random_seed = 2018

                              )

    model.fit(X_train, y_train, eval_set=(X_val, y_val),use_best_model=True)

    
    xx_pred = model.predict_proba(X_val)[:,1].reshape((X_val.shape[0],1))
    

    num+=1

    apscore.append(average_precision_score(y_val,xx_pred,average='weighted'))
    test_ypre = model.predict_proba(X_test)[:,1].reshape((X_test.shape[0],1))
    
    if num == 1:
        cv_pred = np.array(test_ypre).reshape(-1, 1)
    else:
        cv_pred = np.hstack((cv_pred, test_ypre.reshape(-1, 1)))
    
#     prob['prob_'+str(num)]=np.array(test_ypre)
#     evals_ypre = bst.predict(train_x.loc[evals_index],num_iteration = best_iter)
#     cut,best_score = searchBestCut(train_y.loc[evals_index],evals_ypre)
#     fscore.append(best_score)
#     cate['cate_'+str(num)] = (pd.Series(test_ypre)>=cut).astype(int)
#     print(pd.Series(bst.feature_importance(),index=train_x.columns).sort_values(ascending=False).head(10))
#     print('\n')
print(apscore)
print(sum(apscore)/5)
# print(fscore)
# print(sum(fscore)/5)
# res['Pred'] = prob.mean(1)
# res['Class'] = (cate.mean(1)>=0.6).astype(int)
# res.to_csv('result/Test_Pred_Class2.csv',index=False)

(256327, 38)
(28480, 38)


0:	test: 0.9557861	best: 0.9557861 (0)	total: 182ms	remaining: 1m 31s
50:	test: 0.9768442	best: 0.9768442 (50)	total: 15.2s	remaining: 2m 13s
100:	test: 0.9786028	best: 0.9793894 (84)	total: 28.6s	remaining: 1m 53s
150:	test: 0.9772216	best: 0.9793894 (84)	total: 42.5s	remaining: 1m 38s
200:	test: 0.9791262	best: 0.9799234 (185)	total: 56.1s	remaining: 1m 23s
250:	test: 0.9789227	best: 0.9799234 (185)	total: 1m 11s	remaining: 1m 10s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.9799233899
bestIteration = 185

Shrink model to first 186 iterations.


0:	test: 0.9264799	best: 0.9264799 (0)	total: 276ms	remaining: 2m 17s
50:	test: 0.9681660	best: 0.9683816 (49)	total: 14s	remaining: 2m 3s
100:	test: 0.9808358	best: 0.9812901 (96)	total: 26.3s	remaining: 1m 44s
150:	test: 0.9838321	best: 0.9839219 (149)	total: 39.1s	remaining: 1m 30s
200:	test: 0.9840470	best: 0.9847364 (180)	total: 53.1s	remaining: 1m 18s
250:	test: 0.9820076	best: 0.9847364 (180)	total: 1m 7s	remaining: 1m 7s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.9847363842
bestIteration = 180

Shrink model to first 181 iterations.


0:	test: 0.9323340	best: 0.9323340 (0)	total: 447ms	remaining: 3m 43s
50:	test: 0.9843015	best: 0.9850813 (35)	total: 14.1s	remaining: 2m 3s
100:	test: 0.9818963	best: 0.9850813 (35)	total: 26.7s	remaining: 1m 45s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.9850812986
bestIteration = 35

Shrink model to first 36 iterations.


0:	test: 0.9337383	best: 0.9337383 (0)	total: 180ms	remaining: 1m 29s
50:	test: 0.9649165	best: 0.9653795 (47)	total: 14.3s	remaining: 2m 5s
100:	test: 0.9701626	best: 0.9704513 (99)	total: 27.3s	remaining: 1m 47s
150:	test: 0.9729506	best: 0.9743195 (130)	total: 41.7s	remaining: 1m 36s
200:	test: 0.9748953	best: 0.9758527 (181)	total: 56.3s	remaining: 1m 23s
250:	test: 0.9749863	best: 0.9758527 (181)	total: 1m 10s	remaining: 1m 10s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.9758527446
bestIteration = 181

Shrink model to first 182 iterations.


0:	test: 0.9461038	best: 0.9461038 (0)	total: 185ms	remaining: 1m 32s
50:	test: 0.9818025	best: 0.9818025 (50)	total: 14s	remaining: 2m 3s
100:	test: 0.9826217	best: 0.9835146 (65)	total: 29.2s	remaining: 1m 55s
150:	test: 0.9834255	best: 0.9835146 (65)	total: 42.6s	remaining: 1m 38s
200:	test: 0.9835954	best: 0.9839422 (188)	total: 56.3s	remaining: 1m 23s
250:	test: 0.9845990	best: 0.9848060 (242)	total: 1m 10s	remaining: 1m 9s
300:	test: 0.9853705	best: 0.9854200 (297)	total: 1m 23s	remaining: 55.4s
350:	test: 0.9859074	best: 0.9860637 (327)	total: 1m 37s	remaining: 41.5s
400:	test: 0.9859225	best: 0.9865471 (390)	total: 1m 51s	remaining: 27.6s
450:	test: 0.9860717	best: 0.9865471 (390)	total: 2m 5s	remaining: 13.6s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.9865470942
bestIteration = 390

Shrink model to first 391 iterations.
[0.8982797682150906, 0.8250267642221921, 0.830060000314953, 0.8331338115158007, 0.929535631916322]
0.8632071952368717


In [4]:
np.array(test_ypre)

array([[1.45593717e-04],
       [4.32358783e-04],
       [8.06488765e-05],
       ...,
       [2.48775402e-05],
       [1.45198508e-05],
       [2.12646361e-05]])

In [8]:
cv_pred.shape

(28480, 5)

In [10]:
temp = pd.DataFrame(cv_pred)

In [11]:
temp

,0,1,2,3,4
0,0.000146,0.000160,0.000290,0.000059,0.000109
1,0.000432,0.000096,0.000740,0.000328,0.000949
2,0.000081,0.000213,0.000596,0.000125,0.000102
3,0.000068,0.000051,0.000214,0.000033,0.000027
4,0.000083,0.000069,0.000371,0.000123,0.000077
5,0.000024,0.000015,0.000177,0.000015,0.000004
6,0.000015,0.000016,0.000131,0.000036,0.000012
7,0.000021,0.000013,0.000141,0.000035,0.000015
8,0.000076,0.000017,0.000166,0.000040,0.000008
9,0.001407,0.000348,0.005126,0.001436,0.000820


In [12]:
temp.to_csv('Test_Pred_Class2.csv',index=False)